In [ ]:
#Téléchargement des packages Python/spaCy
#!python -m spacy download fr_core_news_sm
import time
start = time.time()
#Importation des packages Python/spaCy
import spacy
end = time.time()
print(end - start)
import re
import pandas as pd
import csv
from langdetect import detect

In [2]:
# open the CSV file using the csv module
with open('GooglePlayComments.csv', newline='',  encoding='utf-8') as csvfile:
    # create a csv reader object
    reader = csv.reader(csvfile)
    # create an empty list to store the strings
    string_list = []
    # loop through each row in the CSV file
    for row in reader:
        # add the string to the list
        #print(row)
        #if detect(row[0]) == 'fr':
        string_list.append(row[0])

#print(len(string_list))

#print the list of strings
#print(string_list)

string_list_clean = []


#Exemple de texte
text = "Selon Wikipédia il l'appelle Laura 😊😊😊(hezignborzingiozrnoivznioznvionzo), La ponctuation est génial l’ensemble des signes qui, dans l’écrit, marquent les divisions et les liaisons des phrases et des membres 'de phrase'. Vous pouvez en apprendre plus sur \"la ponctuation\" en visitant ce site: https://www.larousse.fr/dictionnaires/francais/ponctuation/63717 parce que ce lien donne beaucoup d'informations intéressantes. 😊"
#text = "Fonctionnalités, salut car beau temps aujourd'hui"

#Etapes du texte Preprocessing
nlp = spacy.load("fr_core_news_sm")
listStopWords = [str(x) for x in nlp.Defaults.stop_words]

#Tokenisation
def tokenisation(text):
    return nlp(text)

#Affichage des tokens
def affichage_tokens(doc):
    for token in doc:
        # Print the text and the predicted part-of-speech tag
        print(token.text, token.pos_)

#Removal of explanations
def explanationsRemoval(doc):
    sentence = doc.text
    # Itérer à travers chaque token dans le document
    for token in doc:
        # Si le token est "parce que", "car" ou "puisque", supprimer tous les tokens suivants dans la phrase
        if token.text.lower() in ["parce", "car", "puisque"]:
            sentence = sentence[:token.idx]
            break
    # Supprimer tout ce qui est entre parenthèses
    sentence = re.sub(r'\([^()]*\)', '', sentence)
    return sentence

#Removal of quotes
def quotesRemoval(doc):
    filtered_text = ""
    Punc_list = ['"', "'"]
    boolean_punct = False
    for token in doc:
        if not token.text in Punc_list:
            if not boolean_punct:
                filtered_text += token.text
                filtered_text += " "
        else :
            if boolean_punct :
                boolean_punct = False
            else :
                boolean_punct = True
    return filtered_text

#Removal of URLs
def urlRemoval(text):
    return re.sub('http[s]?://\S+', '', text)

#Removal of contact information
def contactRemoval(text):
    # Regular expressions for phone numbers and email addresses
    phone_regex = r"(?<!\d)(?:\d[ -/\\_\d]*){10}(?!\d)"
    email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  # matches john.doe@example.com

    # Find phone numbers and email addresses in the sentence
    text = re.sub(phone_regex,"", text)
    text = re.sub(email_regex,"", text)

    return text

#Removal of symbols
def symbolsRemoval(text):
    return re.sub(r'[^\w]', ' ', text)

#Removal of stop words
def stopWordsRemoval(text):
    splitext = text.split(" ")
    for word in splitext:
        if word in listStopWords:
            splitext.remove(word)
    return ' '.join(splitext)

def pronounAndDetRemoval(doc):
    #texte = doc.text
    sentence = ""
    for token in doc:
        if (token.pos_ == "PRON" or token.pos_ == "DET"):
            continue
        else:
            sentence += token.text_with_ws
            sentence += " "
    return sentence

def propNounRemoval(doc):
    result = []
    for token in doc:
        if not (token.ent_type_ == "ORG" or token.ent_type_ == "PERSON" or token.text.istitle()):
            result.append(token.text)
    return " ".join(result)

def united(text):
    doc = nlp(text)

    sentence = doc.text
    # Itérer à travers chaque token dans le document
    for token in doc:

        #Remove explanations
        # Si le token est "parce que", "car" ou "puisque", supprimer tous les tokens suivants dans la phrase
        if token.text.lower() in ["parce", "car", "puisque"]:
            sentence = sentence[:token.idx]
            break
    # Supprimer tout ce qui est entre parenthèses
    sentence = re.sub(r'\([^()]*\)', '', sentence)

    doc = nlp(sentence)

    #Filter
    filtered_text = ""
    Punc_list = ['"', "'"]
    boolean_punct = False
    for token in doc:
        if not token.text in Punc_list:
            if not boolean_punct:
                filtered_text += token.text
                filtered_text += " "
        else :
            if boolean_punct :
                boolean_punct = False
            else :
                boolean_punct = True

    sentence = filtered_text
    sentence = re.sub('http[s]?://\S+', '', sentence)

    # Regular expressions for phone numbers and email addresses
    phone_regex = r"(?<!\d)(?:\d[ -/\\_\d]*){10}(?!\d)"
    email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  # matches john.doe@example.com

    # Find phone numbers and email addresses in the sentence
    sentence = re.sub(phone_regex,"", sentence)
    sentence = re.sub(email_regex,"", sentence)
    sentence = re.sub(r'[^\w]', ' ', sentence)
    splitext = sentence.split(" ")
    for word in splitext:
        if word in listStopWords:
            splitext.remove(word)
    sentence= ' '.join(splitext)

    doc = nlp(sentence)
    #texte = doc.text
    text = ""
    for token in doc:
        if (token.pos_ == "PRON" or token.pos_ == "DET"):
            continue
        else:
            text += token.text_with_ws
            text += " "
    sentence = text

    doc = nlp(sentence)

    result = []
    for token in doc:
        if not (token.ent_type_ == "ORG" or token.ent_type_ == "PERSON" or token.text.istitle()):
            result.append(token.text)
    sentence =  " ".join(result)

    return sentence

"""
for string in string_list:
    united_string = united(string)
    string_list_clean.append(united_string)
print(string_list_clean)
#run
#print(united(text))"""
united_string = united(string_list[0])
print(united_string)



                dizaine   jours      sorties     pied      n     accès   données   synthèse   proposé     application      d     allure   moyenne      plus   rythme   moyen      plus   de   vitesse   moyenne         explication      solution        bonne   application      manque   mode   fractionné        ergonomique   très   intuitive          


In [6]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
text = 'はいあぃがとう.'
doc = nlp(text)
print(doc._.language)

{'language': 'ja', 'score': 0.9999999999991745}


#Part 2


In [3]:
united_string

'                dizaine   jours      sorties     pied      n     accès   données   synthèse   proposé     application      d     allure   moyenne      plus   rythme   moyen      plus   de   vitesse   moyenne         explication      solution        bonne   application      manque   mode   fractionné        ergonomique   très   intuitive          '

In [10]:
doc = nlp(united_string)
print(doc._.language)

{'language': 'fr', 'score': 0.9999953794949574}


In [11]:
for token in doc:
    print(token.text)

                
dizaine
  
jours
     
sorties
    
pied
     
n
    
accès
  
données
  
synthèse
  
proposé
    
application
     
d
    
allure
  
moyenne
     
plus
  
rythme
  
moyen
     
plus
  
de
  
vitesse
  
moyenne
        
explication
     
solution
       
bonne
  
application
     
manque
  
mode
  
fractionné
       
ergonomique
  
très
  
intuitive
         


In [19]:
# Define list of POS patterns to detect
patterns = [
    ("noun noun", ["NOUN", "NOUN"]),
    ("verb noun", ["VERB", "NOUN"]),
    ("adjective noun", ["ADJ", "NOUN"]),
    ("noun conjunction", ["NOUN", "CONJ"]),
    ("noun adjective", ["NOUN", "ADJ"]),
    ("noun noun noun", ["NOUN", "NOUN", "NOUN"]),
    ("verb pronoun noun", ["VERB", "PRON", "NOUN"]),
    ("verb noun noun", ["VERB", "NOUN", "NOUN"]),
    ("verb adjective noun", ["VERB", "ADJ", "NOUN"]),
    ("adjective adjective noun", ["ADJ", "ADJ", "NOUN"]),
    ("noun preposition noun", ["NOUN", "ADP", "NOUN"]),
    ("verb determiner noun", ["VERB", "DET", "NOUN"]),
    ("verb noun preposition noun", ["VERB", "NOUN", "ADP", "NOUN"]),
    ("adjective noun noun noun", ["ADJ", "NOUN", "NOUN", "NOUN"]),
    ("adjective conjunction adjective", ["ADJ", "CONJ", "ADJ"]),
    ("verb preposition adjective noun", ["VERB", "ADP", "ADJ", "NOUN"]),
    ("verb pronoun adjective noun", ["VERB", "PRON", "ADJ", "NOUN"]),
    ("noun conjunction noun noun", ["NOUN", "CONJ", "NOUN", "NOUN"]),
]

In [17]:
# Iterate through the reference patterns and check if there is a match in the sentence
for pattern, pos_tags_pattern in patterns:
    if len(pos_tags_pattern) != len(doc):
        continue  # Skip patterns with different number of words
    match = True
    for i in range(len(doc)):
        if pos_tags_pattern[i] != doc[i].pos_:
            match = False
            break
    if match:
        print(f"Detected pattern '{pattern}' in sentence '{united_string}'")
        break


In [31]:
# Define the input sentence
sentence = "The cat sat on the mat"

# Process the sentence with spaCy
doc = nlp(sentence)

# Iterate through the reference patterns and check if there is a match in the sentence
for pattern, pos_tags_pattern in patterns:
    if len(pos_tags_pattern) != len(doc):
        continue  # Skip patterns with different number of words
    match = True
    for i in range(len(doc)):
        if pos_tags_pattern[i] != doc[i].pos_:
            match = False
            break
    if match:
        print(f"Detected pattern '{pattern}' in sentence '{sentence}'")
        break


In [43]:
sentenceByPattern = [[] for i in range((18))]

In [38]:
sentence = "cat dog sat on mat"
doc = nlp(sentence)

In [45]:
for i in range(len(doc) - 3):
    if doc[i].pos_ == "NOUN":
        if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "NOUN":
            if doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[17].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            else:
                sentenceByPattern[4].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
        elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[11].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "NOUN":
            if doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[6].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
            else:
                sentenceByPattern[1].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
    elif doc[i].pos_ == "VERB":
        if doc[i + 1].pos_ == "PRON":
            if doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[17].append(doc[i].text + doc[i + 1].text + doc[i + 2].text + doc[i + 3].text)
                break
            if doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[7].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
            sentenceByPattern[16].append(doc[i].text + doc[i + 1].text + doc[i + 2].text + doc[i + 3].text)
        elif doc[i + 1].pos_ == "NOUN" :
            if doc[i + 2].pos_ == "PREP" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[13].append(doc[i].text + doc[i + 1].text + doc[i + 2].text + doc[i + 3].text)
                break
            elif doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[8].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
                break
            else:
                sentenceByPattern[2].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "DET" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[12].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[9].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
            break
    elif doc[i].pos_ == "ADJ" :
        if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "ADJ":
            sentenceByPattern[15].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "NOUN":
            if doc[i + 2].pos_ == "NOUN":
                if doc[i + 3].pos_ == "NOUN":
                    sentenceByPattern[14].append(doc[i].text + doc[i + 1].text + doc[i + 2].text + doc[i + 3].text)
                    break
                else:
                    sentenceByPattern[5].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
                    break
            else:
                sentenceByPattern[3].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[10].append(doc[i].text + doc[i + 1].text + doc[i + 2].text)
            break
    else:
        break

In [46]:
sentenceByPattern

[[],
 [],
 ['dogsaton'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]